# 검색 엔진 성능 평가


> PC - 상품 : 한줄에 5개씩 2줄 (한페이지 총 40개)

> PC - 한약재 : 세로로 첫 화면에 7개(7.5개) 페이지 내에 총 60개


P@K (Precision at k)
: k개의 상위 검색 결과 중 정답과 연관된 결과가 몇 개가 있는 지 나타내는 지표 (1에 가까울수록 좋음!)

> k = 10으로 설정!

precision (정밀도) : Positive로 예상한 결과 중, 실제로도 Positive인 비율 
= TP / ( TP + FP ) 

recall (재현율) : Positive로 나온 결과 중, Positive로 예측했던 값의 비율
= TP / ( TP + FN )


검색엔진 성능평가에 사용하는 기준으로 맞추면,

- TP : 데이터셋의 검색어를 입력했을 때 실제로도 검색 결과로 제대로 나온 경우

- FP : 해당 데이터셋의 검색어를 입력했을 때 검색 결과 나오지 않은 경우

- FN : 데이터셋에는 있지만 검색어로 입력했을 때 검색 결과 나오지 않은 경우

> F1 - score = 2 * (precision * recall) / ( precision + recall ) 

# 0. 필요한 라이브러리 불러오기

In [1]:
!pip install meilisearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import random
import json

import meilisearch

# 1. 파일 불러오기 및 사용 할 데이터 정리

In [3]:
'''
# updated_testset.csv 파일 불러오기
path = '/content/drive/MyDrive/기업연계 프로젝트/단어사전/data'

data = pd.read_csv(path+'/updated_testset.csv')
data.columns = ['단어','좌문맥ID', '우문맥ID', '가중치(단어비용)', '품사태그', '의미분류','종성 유무','읽기', '타입', '첫 품사', '마지막 품사','표현']

# 잘 불러왔는지 확인
data.tail(3)

'''



# 최종 성능평가에 사용될 데이터 불러오기
testset_data = pd.read_csv('/content/drive/MyDrive/기업연계 프로젝트/test_keyword_1616.csv')
testset = testset_data['keyword']

# 전체 product 데이터 불러오기
all_product_json = open('/content/drive/MyDrive/기업연계 프로젝트/product_all.json', encoding = 'utf-8')
all_product_dict = json.load(all_product_json)
total_search_result = pd.DataFrame(all_product_dict)

In [ ]:
'''
## OCR 사전 중 랜덤으로 추출한 100개의 성능평가 테스트셋
# -> 문제점: 사전에 있는 단어더라도 'Product' 데이터에 없는 경우가 너무 많아 성능평가 결과가 무의미..
words = []

for x in data['단어']:
  words.append(x)

OCR_word_list = random.sample(words,100)


## 위의 문제점으로 OCR 데이터에서 단어를 추출하되, 그 중 몇 개의 단어를 선정하여 다양한 변주를 주어 비교하기로 결정 
# 사용할 리스트 설정 및 초기화
soyosan_word = []
daebotang_word = []
gamibang_word = []
makmundong_word = []
illness_word = []
pain_word = []

soyosan_list = []
daebotang_list = []
gamibang_list = []
makmundong_list = []
illness_list = []
pain_list = []


# 사용 단어 추출 및 저장
soyosan = data[(data['단어'].str.contains('소요산'))]['단어'] 
daebotang = data[(data['단어'].str.contains('대보탕'))]['단어']
gamibang = data[(data['단어'].str.contains('가미방'))]['단어']

makmundong = data[(data['단어'].str.contains('맥문동'))]['단어']
illness = data[(data['단어'].str.contains('질환'))]['단어']
pain = data[(data['단어'].str.contains('통증'))]['단어']


def add_word(word_name,word_list):
  for i in word_name:
    word_list.append(i)

  return word_list

soyosan_list = add_word(soyosan,soyosan_word)
daebotang_list = add_word(daebotang,daebotang_word)
gamibang_list = add_word(gamibang,gamibang_word)
makmundong_list = add_word(makmundong,makmundong_word)
illness_list = add_word(illness,illness_word)
pain_list = add_word(pain,pain_word)


# 각 단어에 대한 다양한 형태들 추가 입력 
soyosan_plus = ['가미 소요산', '단치 소요산', '인삼 소요산', '청간 소요산']
daebotang_plus = ['가미 대보탕', '녹각 대보탕', '녹용 대보탕', '십전 대보탕']
gamibang_plus = ['강활속단탕 가미방','공진단 가미방','방풍통성산 가미방','사육탕 가미방','생간건비탕 가미방','억간산 가미방','정전가미이진탕 가미방','청간탕 가미방','평진탕 가미방','형방사백산 가미방']
makmundong_plus = ['소엽맥문동', '소엽 맥문동']
illness_plus = ['관상동맥 질환','관절 질환','낙설 질환','내이 질환','뇌간혈관 질환','비뇨기계 질환','소화기 질환','역류 질환','외이도 질환','운동 질환','유방 질환','이관 질환','일차두통 질환','중이 질환','탈수초성 질환','피부 질환','항문 질환','호흡기 질환','후두 질환']
pain_plus = ['근막통증 증후군','근막 통증 증후군' ,'다리 통증','만성 통증','비기질적 통증','슬개대퇴 통증','슬개대퇴 통증증후군','슬개대퇴통증 증후군','어깨 통증','위성 통증','일차 통증','종아리 통증','중앙 통증','중족 골통증','척추 통증','통증 양상','통증 유발점','통증 유형','하지 통증','허리 통증','혀 통증']

# 각 단어별 데이터셋 지정
soyosan_total = soyosan_list + soyosan_plus
daebotang_total = daebotang_list + daebotang_plus
gamibang_total =  gamibang_list + gamibang_plus
makmundong_total =  makmundong_list + makmundong_plus
illness_total =  illness_list + illness_plus
pain_total =  pain_list + pain_plus
'''

# 2. 클라이언트 연결 및 성능평가 모델 함수 정의

In [4]:
# word_list 검색 (모든 결과 포함 버전)                            (   )
# word_list 검색 (NAME, META_TITLE, META_DESCRIPTION 적용 버전)   ( V )
def search_word(word, ip):

  # client = meilisearch.Client('http://35.216.64.12:7700', 'team10')     # GCP
  # client = meilisearch.Client('http://52.79.49.97:7700', 'team10')     # aws
  client = meilisearch.Client(ip, 'team10') 

  # index_num = int(input('Choose index : \n 1. product \n 2. crawling_data \n 3. articles \n  :'))
  indexes = ['product','crawling_data','articles']
  # word = input('Search Word : ')
  # answer = client.index(indexes[index_num-1]).search(word,{'limit': 20})
  answer = client.index(indexes[0]).search(word,{'limit': 20})

  return answer


# 검색결과 데이터 프레임으로 저장
def search_result(answer,word):
  global temp, columns

  temp = []
  columns = ['name','meta_title','meta_description']

  for x in range(len(answer['hits'])):
    temp.append([answer['hits'][x]['name'], answer['hits'][x]['meta_title'], answer['hits'][x]['meta_description']])


  globals()[f'df_{word}'] = pd.DataFrame(data = temp, columns = columns)
  result = globals()[f'df_{word}']
  return result


# P@K (Precision at k) 검색 엔진 성능 평가 함수
def calculate_PnK(result,word):
  global  k_num, count_O, count_X
  # 상위 검색어 수(k) 지정
  k_num = 10

  # count 변수 설정 
  count_O = 0
  count_X = 0

  for x in range(k_num):

    # 검색결과가 k보다 적을 경우
    if len(result) < k_num:
      while count_O + count_X < len(result):
        if (word in result.iloc[x,0]) | (word in result.iloc[x,1]) | (word in result.iloc[x,2]):
          count_O += 1
        elif (word not in result.iloc[x,0]) | (word not in result.iloc[x,1]) | (word not in result.iloc[x,2]):
          count_X += 1
        else:
          print('error')
          break

    # 검색 결과가 없는 경우
    elif len(result) == 0:
      print('No search result!')
      break

    # 검색 결과가 k보다 많은 경우
    else:
      if (word in result.iloc[x,0]) | (word in result.iloc[x,1]) | (word in result.iloc[x,2]):
        count_O += 1
      elif (word not in result.iloc[x,0]) | (word not in result.iloc[x,1]) | (word not in result.iloc[x,2]):
        count_X += 1
      else:
        print('error')
        break
  
  if (count_O + count_X ) == 0 :
    PnK = 'no search result'
  elif (count_O + count_X ) != 0:
    PnK = count_O / (count_O + count_X )

  return PnK


# F1 - score 검색 엔진 성능 평가 함수
def calculate_F1score(result,word):
  global  precision, recall

  # 변수 초기화 
  count = 0
  TP = 0
  FP = 0
  FN = 0

  for x in range(len(result)):

    # 검색 결과가 없는 경우
    if len(result) == 0:
      print('No seearch result!')
      break

    else:
      # 데이터셋의 검색어를 입력했을 때 실제로도 검색 결과로 제대로 나온 경우
      if (word in result.iloc[x,0]) | (word in result.iloc[x,1]) | (word in result.iloc[x,2]):
        TP += 1

      # 검색어가 들어있지 않지만 검색 결과로 나온 경우
      elif (word not in result.iloc[x,0]) | (word not in result.iloc[x,1]) | (word not in result.iloc[x,2]):
        FP += 1

      # 데이터셋의 검색어를 입력했을 때 검색 결과 나오지 않은 경우
      elif (word in total_search_result) & ((word not in result.iloc[x,0]) | (word not in result.iloc[x,1]) | (word not in result.iloc[x,2])):
        FN += 1 

      else:
        print('error')
        break
  

  if TP + FP == 0 or TP + FN == 0 :
    F1score = 'no search result'
  elif TP + FP != 0 or TP + FN != 0:
    precision = TP / ( TP + FP )
    recall =  TP / ( TP + FN )

    if precision + recall == 0:
      F1score = 'precision and recall == 0'
    elif precision + recall != 0:
      F1score = 2 * (precision * recall) / ( precision + recall )

  return F1score

# 3. P@k & F1 - score 계산

In [5]:
## 사용할 검색어 리스트 선택
# OCR_word_list
# soyosan_total
# daebotang_total
# gamibang_total

# makmundong_total
# illness_total
# pain_total

# testset

word_list = testset

In [12]:
PnK_score = []
F1 = []
precision_score = []
recall_score = []

AWS = 'http://52.79.49.97:7700'
GCP = 'http://35.216.64.12:7700'
Vainilla = 'http://192.168.0.12:7700'


for word in word_list:
  answer = search_word(word,AWS)  # 사용하고자 하는 검색 환경 선택 (GCP or AWS or Vainilla)
  result = search_result(answer,word)

  # P@k
  PnK = calculate_PnK(result,word)
  #display(calculate_PnK(result,word))
  PnK_score.append(PnK)

  # F1 - score
  F1_score = calculate_F1score(result,word)
  #display(calculate_F1score(result,word))
  F1.append(F1_score)
  precision_score.append(precision)
  recall_score.append(recall)

## calculate result

### testset test result - P@k & F1 score

In [7]:
# GCP
conclusion = pd.DataFrame(data = PnK_score,index = word_list, columns = ['P@k (precision at k)'])
conclusion['F1 - score'] = F1
conclusion['precision (정밀도)'] = precision_score
conclusion['recall (재현율)'] = recall_score
display(conclusion)
print('\n'+'<<  P@k (precision at k)  >>')
print(conclusion['P@k (precision at k)'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  F1 - score  >>')
print(conclusion['F1 - score'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  precision (정밀도)  >>')
print(conclusion['precision (정밀도)'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  recall (재현율)  >>')
print(conclusion['recall (재현율)'].value_counts() )

GCP_conclusion = conclusion
GCP_conclusion.to_csv('GCP_테스트결과.csv', index = False)

,P@k (precision at k),F1 - score,precision (정밀도),recall (재현율)
keyword,,,,
한방내과,0.7,0.518519,0.350000,1.0
진찰진단,0.1,0.095238,0.050000,1.0
가이드라인,0.4,0.333333,0.200000,1.0
동맥경화증,0.1,0.095238,0.050000,1.0
파이오니어,1.0,0.571429,0.400000,1.0
...,...,...,...,...
임상족부학과,0.1,0.095238,0.050000,1.0
족부보조기,0.1,0.125,0.066667,1.0
액티베이터건,0.1,0.095238,0.050000,1.0



<<  P@k (precision at k)  >>
1.0                 576
0.1                 559
0.2                 192
0.3                  94
0.4                  57
0.5                  42
0.6                  28
0.7                  21
0.0                  17
0.8                  15
0.9                  12
no search result      3
Name: P@k (precision at k), dtype: int64 

~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~

<<  F1 - score  >>
0.095238    447
1.000000    217
0.181818    175
0.400000     85
0.333333     78
           ... 
0.592593      1
0.588235      1
0.421053      1
0.782609      1
0.476190      1
Name: F1 - score, Length: 80, dtype: int64 

~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.

In [13]:
# AWS
conclusion = pd.DataFrame(data = PnK_score,index = word_list, columns = ['P@k (precision at k)'])
conclusion['F1 - score'] = F1
conclusion['precision (정밀도)'] = precision_score
conclusion['recall (재현율)'] = recall_score
display(conclusion)
print('\n'+'<<  P@k (precision at k)  >>')
print(conclusion['P@k (precision at k)'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  F1 - score  >>')
print(conclusion['F1 - score'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  precision (정밀도)  >>')
print(conclusion['precision (정밀도)'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  recall (재현율)  >>')
print(conclusion['recall (재현율)'].value_counts() )

AWS_conclusion = conclusion
AWS_conclusion.to_csv('AWS_테스트결과.csv', index = False)

,P@k (precision at k),F1 - score,precision (정밀도),recall (재현율)
keyword,,,,
한방내과,0.7,0.518519,0.350000,1.0
진찰진단,0.1,0.095238,0.050000,1.0
가이드라인,0.4,0.333333,0.200000,1.0
동맥경화증,1.0,0.222222,0.125000,1.0
파이오니어,1.0,0.571429,0.400000,1.0
...,...,...,...,...
임상족부학과,0.1,0.095238,0.050000,1.0
족부보조기,0.1,0.125,0.066667,1.0
액티베이터건,0.1,0.095238,0.050000,1.0



<<  P@k (precision at k)  >>
1.0                 569
0.1                 565
0.2                 189
0.3                  95
0.4                  58
0.5                  39
0.6                  29
0.0                  20
0.7                  19
0.8                  17
0.9                  12
no search result      4
Name: P@k (precision at k), dtype: int64 

~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~

<<  F1 - score  >>
0.095238    459
1.000000    219
0.181818    173
0.333333     92
0.400000     75
           ... 
0.588235      1
0.782609      1
0.689655      1
0.761905      1
0.476190      1
Name: F1 - score, Length: 78, dtype: int64 

~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.

In [ ]:
# Vainilla
conclusion = pd.DataFrame(data = PnK_score,index = word_list, columns = ['P@k (precision at k)'])
conclusion['F1 - score'] = F1
conclusion['precision (정밀도)'] = precision_score
conclusion['recall (재현율)'] = recall_score
display(conclusion)
print('\n'+'<<  P@k (precision at k)  >>')
print(conclusion['P@k (precision at k)'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  F1 - score  >>')
print(conclusion['F1 - score'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  precision (정밀도)  >>')
print(conclusion['precision (정밀도)'].value_counts(),'\n'*2+'~.'*100+'~' )
print('\n'+'<<  recall (재현율)  >>')
print(conclusion['recall (재현율)'].value_counts() )

Vainilla_conclusion = conclusion
Vainilla_conclusion.to_csv('바닐라_메일리서치_테스트결과.csv', index = False)

# + code test

### P@k test

P@K (Precision at k)
: k개의 상위 검색 결과 중 정답과 연관된 결과가 몇 개가 있는 지 나타내는 지표 (1에 가까울수록 좋음!)




> PC - 상품 : 한줄에 5개씩 한페이지 총 40개

> PC - 한약재 : 세로로 첫 화면에 7개(7.5개) 페이지 내에 총 60개

In [ ]:
# P@K (Precision at k) 검색 엔진 성능 평가 함수
def calculate_PnK(result,word):
  global  k_num, count_O, count_X
  # 상위 검색어 수(k) 지정
  k_num = 5

  # count 변수 설정 
  count_O = 0
  count_X = 0

  for x in range(k_num):

    # 검색결과가 k보다 적을 경우
    if len(result) < k_num:
      while count_O + count_X < len(result):
        if word in result.iloc[x,0]:
          count_O += 1
        elif word not in result.iloc[x,0]:
          count_X += 1
        else:
          print('error')
          break

    # 검색 결과가 없는 경우
    elif len(result) == 0:
      print('No seearch result!')
      break

    # 검색 결과가 k보다 많은 경우
    else:
      if word in result.iloc[x,0]:
        count_O += 1
      elif word not in result.iloc[x,0]:
        count_X += 1
      else:
        print('error')
        break

  P_k = count_O / (count_O + count_X )
  print(P_k)

  calculate_PnK(result)

In [ ]:
temp = ['작약산을 검색했을때 나오는 결과','여기도 빈내용','작약 당귀산','가미당귀작약산을 포함한 문장','작약산 당귀', '아무 내용도 아님','당귀작약만 포함 산은 안붙음',' 작약탕은 나옴?']

word_list = ['작약산','대보탕']

ex_1 = pd.DataFrame(data = temp, columns = ['작약산'])
result = ex_1

In [ ]:
k_num = 5

count_O = 0
count_X = 0

for x in range(k_num):

  if len(ex_1) < k_num:
    while count_O + count_X < len(ex_1):
      if word_list[0] in ex_1.iloc[x,0]:
        count_O += 1
      elif word_list[0] not in ex_1.iloc[x,0]:
        count_X += 1
      else:
        print('error')
        break

  elif len(ex_1) == 0:
    print('No seearch result!')
    break

  else:
    if word_list[0] in ex_1.iloc[x,0]:
      count_O += 1
    elif word_list[0] not in ex_1.iloc[x,0]:
      count_X += 1
    else:
      print('error')
      break

P_k = count_O / (count_O + count_X )
print(P_k)

0.6


In [ ]:
print(count_O,count_X)

4 1


### F1 - score test

precision (정밀도) : Positive로 예상한 결과 중, 실제로도 Positive인 비율 
= TP / ( TP + FP ) 

recall (재현율) : Positive로 나온 결과 중, Positive로 예측했던 값의 비율
= TP / ( TP + FN )


검색엔진 성능평가에 사용하는 기준으로 맞추면,

- TP : 데이터셋의 검색어를 입력했을 때 실제로도 검색 결과로 제대로 나온 경우

- FP : 데이터셋의 검색어를 입력했을 때 검색 결과 나오지 않은 경우

- FN : 검색어가 들어있지 않지만 검색 결과로 나온 경우

> F1 - score = 2 * (precision * recall) / ( precision + recall ) 

In [ ]:
# 전체 product 데이터 불러오기
all_product_json = open('/content/drive/MyDrive/기업연계 프로젝트/product_all.json', encoding = 'utf-8')
all_product_dict = json.load(all_product_json)
total_search_result = pd.DataFrame(all_product_dict)
total_search_result

,_id,date_created,date_updated,images,dimensions,name,description,meta_description,meta_title,tags,...,date_stock_expected,date_sale_from,date_sale_to,stock_tracking,stock_preorder,stock_backorder,category_id,category_ids,options,variants
0,{'$oid': '5d13115e32026c0b35383897'},{'$date': '2019-06-26T06:31:58.090Z'},{'$date': '2022-09-06T01:45:12.198Z'},"[{'id': {'$oid': '5d1311f532026c0b35383898'}, ...","{'length': 0, 'width': 0, 'height': 0}",KCD 한방내과 진찰진단 가이드라인,"<h2 align=""left""><strong>Medistream</strong><s...",진료에 어려움 있을 때 먼저 찾는 책,KCD 한방내과 진찰진단 가이드라인,[가온해미디어],...,None,{'$date': '2019-06-25T15:00:00Z'},{'$date': '2021-12-30T15:00:00Z'},False,False,False,{'$oid': '5cf8bbba0098b2225c5dfaa3'},"[{'$oid': '5e5d8beabfe4260944782c14'}, {'$oid'...","[{'id': {'$oid': '5d398a464e77525ec5ca1031'}, ...","[{'id': {'$oid': '5d8324ce3f0e6805c47066f0'}, ..."
1,{'$oid': '5dd4ad4e2bb59605ca3d02fa'},{'$date': '2019-11-20T03:04:46.556Z'},{'$date': '2022-01-28T04:33:53.149Z'},"[{'id': {'$oid': '5dd4c8bc2bb59605ca3d031c'}, ...","{'length': 0, 'width': 0, 'height': 0}",3B 13. 담석 모형 (모델명 : K26),"<p><strong data-stringify-type=""bold""><span st...",담석모형,3B 13. 담석 모형 (모델명 : K26),[],...,None,{'$date': '2019-11-20T05:14:08.041Z'},{'$date': '2105-03-16T15:00:00Z'},False,False,False,{'$oid': '5e5d8f66bfe4260944782c17'},[],"[{'id': {'$oid': '60e47962ae811e631eda7f64'}, ...","[{'id': {'$oid': '60e47962ae811e631eda7f65'}, ..."
2,{'$oid': '5d9edfe70dabe405b156e213'},{'$date': '2019-10-10T07:38:15.467Z'},{'$date': '2022-09-07T04:46:33.117Z'},"[{'id': {'$oid': '5d9ee3320dabe405b156e216'}, ...","{'length': 0, 'width': 0, 'height': 0}",우리 아이 감기,<h2><strong>우리 아이 감기&nbsp;</strong></h2>\n<p><...,약물남용과 과잉치료가 없는 새로운 감기 진료를 목표로,우리 아이 감기,[물고기숲],...,None,{'$date': '2019-10-09T15:00:00Z'},{'$date': '2122-01-24T02:27:22.441Z'},False,False,False,{'$oid': '5cf8bbba0098b2225c5dfaa3'},"[{'$oid': '5e5d8beabfe4260944782c14'}, {'$oid'...","[{'id': {'$oid': '60e47961ae811e631eda7f34'}, ...","[{'id': {'$oid': '60e47961ae811e631eda7f35'}, ..."
3,{'$oid': '5dd4ad512bb59605ca3d0301'},{'$date': '2019-11-20T03:04:49.746Z'},{'$date': '2022-01-28T04:25:27.172Z'},"[{'id': {'$oid': '5dd4c8f72bb59605ca3d0322'}, ...","{'length': 0, 'width': 0, 'height': 0}",3B 20. 동맥경화증 모형(2파트 분리) (모델명 : G40),"<p><strong data-stringify-type=""bold""><span st...",동맥경화증 모형,3B 20. 동맥경화증 모형(2파트 분리) (모델명 : G40),[],...,None,{'$date': '2019-11-20T05:17:20.106Z'},{'$date': '2097-06-16T15:00:00Z'},False,False,False,{'$oid': '5e5d8f66bfe4260944782c17'},[{'$oid': '6064015d0ccc56001bc7a2f0'}],"[{'id': {'$oid': '60e47962ae811e631eda7f79'}, ...","[{'id': {'$oid': '60e47962ae811e631eda7f7a'}, ..."
4,{'$oid': '5de75b5d4267e105dffeebb2'},{'$date': '2019-12-04T07:08:13.220Z'},{'$date': '2022-09-13T00:05:04.778Z'},"[{'id': {'$oid': '5de75c5b4267e105dffeebb3'}, ...","{'length': 0, 'width': 0, 'height': 0}",파이오니어 AST/ALT 혈액검사기 스트립,"<p><strong><span style=""color: #9a3837; backgr...","10T 개당 3,850원. 아이스박스 냉장출고합니다.",파이오니어 AST/ALT 간기능 혈액검사기 스트립,"[혈액검사, 혈검, 파이오니어, 파이오니아, 간기능, 스트립, 말초채혈, 채혈]",...,None,{'$date': '2019-12-03T15:00:00Z'},{'$date': '2022-12-30T15:00:00Z'},True,False,False,{'$oid': '5df6ea414267e105dffef922'},"[{'$oid': '5e5d8beabfe4260944782c14'}, {'$oid'...","[{'id': {'$oid': '60e47962ae811e631eda7f8e'}, ...","[{'id': {'$oid': '60e47962ae811e631eda7f8f'}, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5136,{'$oid': '63200ecf6c2e37002369ee01'},{'$date': '2022-09-13T05:02:07.181Z'},None,[],"{'length': 0, 'width': 0, 'height': 0}",현대교역 엘린 베개 딥옐로우,,엘린 전용베개,현대교역 엘린 베개 진오크,[],...,None,{'$date': '2021-04-26T15:00:00Z'},{'$date': '2099-12-30T15:00:00Z'},False,False,False,{'$oid': '5e5d8fe7bfe4260944782c19'},"[{'$oid': '62ecbae5a670760022a3dbf5'}, {'$oid'...","[{'id': {'$oid': '63200ecf6c2e37002369ee02'}, ...","[{'id': {'$oid': '63200ecf6c2e37002369ee03'}, ..."
5137,{'$oid': '63200f166c2e37002369ee06'},{'$date': '2022-09-13T05:03:18.192Z'},None,[],"{'length': 0, 'width': 0, 'heigh

In [ ]:
'''

# product 전체 데이터
query = ['가','나','다','라','마','바','사','아','자','차','카','파','타','하','0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','n','m','o','p','q','r','s','t','u','v','w','x','y','z']  
  
  
client = meilisearch.Client('http://35.216.64.12:7700', 'team10')
#client = meilisearch.Client('http://52.79.49.97:7700', 'team10')
# index_num = int(input('Choose index : \n 1. product \n 2. crawling_data \n 3. articles \n  :'))
indexes = ['product','crawling_data','articles']
# word = input('Search Word : ')
# answer = client.index(indexes[index_num-1]).search(word,{'limit': 20})

df_search_word_list = []

for x in range(len(query)):
  globals()[f'query_{x}'] = client.index(indexes[0]).search(f'{query[x]}',{'limit': 1000})
  globals()[f'all_answer_{x}'] = pd.DataFrame(globals()[f'query_{x}']['hits'])
  globals()[f'all_answer_{x}'] = globals()[f'all_answer_{x}'][['name','meta_title','meta_description']] 
  df_search_word_list.append(globals()[f'all_answer_{x}'])

# total_search_result = pd.concat([all_answer_0,all_answer_1,all_answer_2,all_answer_3,all_answer_4,all_answer_5,all_answer_6,all_answer_7,all_answer_8,all_answer_9,all_answer_10,all_answer_11,all_answer_12,all_answer_13], ignore_index = True)
total_search_result = pd.concat(df_search_word_list)
total_search_result.drop_duplicates(['name', 'meta_title','meta_description'], keep = 'first', inplace = True)
total_search_result

'''

,name,meta_title,meta_description
0,가슴베개 다크브라운,가슴 베개 다크브라운,피부 무자극 레자 원단
1,동의수세보원가이드,동의수세보원가이드,성명론부터 사상인변증론까지 일관된 해설
2,크린가드 황사마스크 수프림 플러스 KF 94 대형 (25매*4/박스),크린가드 황사마스크 수프림 플러스 KF 94 대형 (25매*4/박스),25매*4/박스
3,하부가림판 데스크 1600 (DSAD116M),하부가림판 데스크 1600 (DSAD116M),DSAD116M
4,하부가림판 데스크 1800 (DSAD118M),하부가림판 데스크 1800 (DSAD118M),DSAD118M
...,...,...,...
0,X,X,
2,X,X,소형 일반 냉장고
45,흉부영상진단X선,흉부영상진단X선,흉부영상의학 교과서
81,대한한의영상학회_근골격계 Xray,근골격계 Xray,


In [ ]:
# F1 - score 검색 엔진 성능 평가 함수
def calculate_F1score(result,word):
  global  TP, FP, FN, count

  # 변수 초기화 
  count = 0
  TP = 0
  FP = 0
  FN = 0

  for x in range(len(result)):

    # 검색 결과가 없는 경우
    if len(result) == 0:
      print('No seearch result!')
      break

    else:
      # 데이터셋의 검색어를 입력했을 때 실제로도 검색 결과로 제대로 나온 경우
      if (word in result.iloc[x,0]) or (word in result.iloc[x,1]) or (word in result.iloc[x,2]):
        TP += 1

      # 검색어가 들어있지 않지만 검색 결과로 나온 경우
      elif (word not in result.iloc[x,0]) or (word not in result.iloc[x,1]) or (word not in result.iloc[x,2]):
        FN += 1

      # 데이터셋의 검색어를 입력했을 때 검색 결과 나오지 않은 경우
      elif (word in total_search_result) & ((word not in result.iloc[x,0]) or (word not in result.iloc[x,1]) or (word not in result.iloc[x,2])):
        FP += 1 

      else:
        print('error')
        break
  
    precision = TP / ( TP + FP )
    recall =  TP / ( TP + FN )

    F1score = 2 * (precision * recall) / ( precision + recall )

  return F1score

In [ ]:
calculate_F1score(result,word)

NameError: ignored

## calculate result - test

### search result - P@k score

#### OCR_word_list

In [ ]:
# aws
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

0.0    100
dtype: int64

,0
활혈탕,0.0
자오가,0.0
발기부전,0.0
봉래,0.0
열극생풍증,0.0
...,...
독활생기탕,0.0
가미대강활탕,0.0
당귀,0.0
금궤신기환,0.0


#### soyosan_total

In [ ]:
# aws
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

no search result    5
0.0                 4
dtype: int64

,0
가미소요산,0.0
단치소요산,no search result
소요산,no search result
인삼소요산,0.0
청간소요산,no search result
가미 소요산,0.0
단치 소요산,no search result
인삼 소요산,0.0
청간 소요산,no search result


In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

#### daebotang_total

In [ ]:
# daebotang_total
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

0.0                 6
no search result    3
dtype: int64

,0
가미대보탕,0.0
녹각대보탕,0.0
녹용대보탕,0.0
십전대보탕,no search result
대보탕,no search result
가미 대보탕,0.0
녹각 대보탕,0.0
녹용 대보탕,0.0
십전 대보탕,no search result


In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

#### gamibang_total

In [ ]:
# gamibang_total
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

0.0                 11
no search result    10
dtype: int64

,0
가미방,no search result
강활속단탕가미방,0.0
공진단가미방,no search result
방풍통성산가미방,0.0
사육탕가미방,no search result
생간건비탕가미방,no search result
억간산가미방,no search result
정전가미이진탕가미방,0.0
청간탕가미방,no search result
평진탕가미방,0.0


In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

#### makmundong_total

In [ ]:
# aws
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

1.0    3
0.0    2
dtype: int64

,0
맥문동,1.0
맥문동원지산,0.0
맥문동탕,1.0
소엽맥문동,1.0
소엽 맥문동,0.0


In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

0.0    2
1.0    2
0.2    1
dtype: int64

,0
맥문동,1.0
맥문동원지산,0.0
맥문동탕,0.2
소엽맥문동,1.0
소엽 맥문동,0.0


#### illness_total

In [ ]:
# illness_total
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

no search result    20
0.0                 16
1.0                  3
dtype: int64

,0
관상동맥질환,no search result
관절질환,0.0
낙설질환,no search result
내이질환,no search result
뇌간혈관질환,no search result
비뇨기계질환,no search result
소화기질환,1.0
역류질환,0.0
외이도질환,no search result
운동질환,0.0


In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

no search result    26
0.0                  9
1.0                  3
0.6                  1
dtype: int64

,0
관상동맥질환,no search result
관절질환,no search result
낙설질환,no search result
내이질환,no search result
뇌간혈관질환,no search result
비뇨기계질환,no search result
소화기질환,1.0
역류질환,no search result
외이도질환,no search result
운동질환,no search result


#### pain_total

In [ ]:
# pain_total
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

0.0                 31
no search result     5
0.2                  4
0.4                  1
1.0                  1
dtype: int64

,0
국제통증연구학회,0.0
근막통증증후군,0.4
다리통증,0.0
만성통증,0.2
비기질적통증,no search result
슬개대퇴통증,0.0
슬개대퇴통증증후군,0.0
어깨통증,0.2
위성통증,no search result
일차통증,0.0


In [ ]:
# GCP
conclusion = pd.DataFrame(PnK_score,word_list)
display(conclusion.value_counts())
conclusion

0.0    42
dtype: int64

,0
국제통증연구학회,0.0
근막통증증후군,0.0
다리통증,0.0
만성통증,0.0
비기질적통증,0.0
슬개대퇴통증,0.0
슬개대퇴통증증후군,0.0
어깨통증,0.0
위성통증,0.0
일차통증,0.0


### search result - F1 score

#### illness_total


In [ ]:
# aws
conclusion = pd.DataFrame(F1,word_list)
display(conclusion.value_counts())
conclusion

In [ ]:
# GCP
conclusion = pd.DataFrame(F1,word_list)
display(conclusion.value_counts())
conclusion

no result              35
1.0                     5
0.33333333333333337     1
0.8                     1
dtype: int64

,0
국제통증연구학회,no result
근막통증증후군,1.0
다리통증,no result
만성통증,0.8
비기질적통증,no result
슬개대퇴통증,no result
슬개대퇴통증증후군,no result
어깨통증,1.0
위성통증,no result
일차통증,no result


#### pain_total


In [ ]:
# aws
conclusion = pd.DataFrame(F1,word_list)
display(conclusion.value_counts())
conclusion

In [ ]:
# GCP
conclusion = pd.DataFrame(F1,word_list)
display(conclusion.value_counts())
conclusion

no result              35
1.0                     5
0.33333333333333337     1
0.8                     1
dtype: int64

,0
국제통증연구학회,no result
근막통증증후군,1.0
다리통증,no result
만성통증,0.8
비기질적통증,no result
슬개대퇴통증,no result
슬개대퇴통증증후군,no result
어깨통증,1.0
위성통증,no result
일차통증,no result
